In [12]:
import os, yaml, sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import h5py
import copy
import pickle
%matplotlib inline

ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])
from data_preprocessing.preprocessing import create_bins, get_firing_rate, convert_gaze_coordinates, cut_excess_timepoints

In [14]:
def format_in_trials(file_list, len_avg_window, rasters, trials, stimuli):
    unique_stimuli_names = set(file_list)
    final_res_neural = {name : [] for name in unique_stimuli_names}
    final_res_gaze = copy.deepcopy(final_res_neural)

    # correctly estimates trials durations
    for idx, fn in enumerate(file_list):  # range(len(stimuli)): 
        # print(stimuli[idx]["trial_number"],stimuli[idx]["trial_number"].shape)
        trial_number = (int(stimuli[idx]["trial_number"][0].item()) - 1)  # extracts the trial number to which the stimulus corresponds (-1 because of python indexing)
        
        if trials[trial_number]["success"] == 1 and stimuli[idx]["filename"] == fn:
            trial_start = stimuli[idx]["start_time"][0].item()
            trial_end = stimuli[idx]["stop_time"][0].item()
            trial_duration = trial_end - trial_start
            #print(fn, trial_duration)
            stim_onset_delay = trial_start - trials[trial_number]["start_time"][0].item()
            stim_onset_delay = int(stim_onset_delay) - 1  # -1 for python indexing
            gaze_signal = trials[trial_number]["eye_data"][0]
            end_gaze = min(stim_onset_delay + int(trial_duration), len(gaze_signal))
            gaze_signal = gaze_signal[stim_onset_delay:end_gaze, :].T # extracts gaze from the stimulus onset till the end of the trial
            trial_start_int = int(trial_start)
            trial_end_int = int(trial_end)
            bins = create_bins(trial_duration, len_avg_window)
            neural_signal = rasters[trial_start_int:trial_end_int, :].T  # slices the trial from raster
            trial_firing_rate = get_firing_rate(bins, neural_signal)
            trial_gaze = get_firing_rate(bins, gaze_signal)
            trial_gaze = convert_gaze_coordinates(trial_gaze)
            final_res_neural[fn].append(trial_firing_rate)
            final_res_gaze[fn].append(trial_gaze)
        # if trials[trial_number]["success"] == 1 and stimuli[idx]["filename"] == fn:
    # end for i in range(len(stimuli)):
    final_res_neural = cut_excess_timepoints(final_res_neural)
    final_res_gaze = cut_excess_timepoints(final_res_gaze)
    return final_res_neural, final_res_gaze


In [61]:
day = "0831"
rasters_path = f"/Volumes/LivingstoneLab/Data/Data-Formatted/red_2025{day}-rasters.h5"
exp_path = f"/Volumes/LivingstoneLab/Data/Data-Formatted/red_2025{day}_experiment.mat"

In [62]:
with h5py.File(rasters_path, "r") as f:
    rasters = f["rasters"][:]
d = loadmat(exp_path)
trials = d["Trials"]
stimuli = d["Stimuli"]


In [70]:
stimuli["subdir"]

array([[array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [array([], dtype='<U1')],
       [ar

In [59]:
s = np.concatenate(stimuli["filename"])
print(s)
# file_list = [str(x[0]) for x in s]


[array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U1') array([], dtype='<U1') array([], dtype='<U1')
 array([], dtype='<U

In [60]:
counter = 0
for i in range(len(trials)):
    print(trials[i]["success"])
    if trials[i]["success"] ==1:
        counter +=1

[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[1]], dtype=uint8)]
[array([[0]], dtype=uint8)]
[array([[0]], dtype=

In [56]:
counter

0

In [22]:
resolution_Hz = 60
s = np.concatenate(stimuli["filename"])
file_list = [str(x[0]) for x in s]
len_window_firing_rate = 1000/resolution_Hz
neural, gaze = format_in_trials(file_list, len_window_firing_rate, rasters, trials, stimuli)


08:41:06 - Warning, YDXJ0096.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0095.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0099.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0094.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0097.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0092.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0087.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0090.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0086.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0096.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0095.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0099.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0094.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0097.MP4 doesn't have any successful trial
08:41:06 - Warning, YDXJ0092.MP4 doesn't have any successful t

In [18]:
pkl_path = "/Volumes/LivingstoneLab/tiziano/data/neural_red_250818_plx_60Hz.pkl"

with open(pkl_path, "rb") as f:
    neural_pkl = pickle.load(f)

print(type(neural_pkl))
# if isinstance(neural_pkl, dict):
#     print("keys:", list(neural_pkl.keys()))
# else:
#     try:
#         print("length:", len(neural_pkl))
#     except Exception:
#         pass

<class 'dict'>


In [19]:
print(neural_pkl)

{'YDXJ0086.MP4': array([], dtype=float64), 'YDXJ0094.MP4': array([], dtype=float64), 'YDXJ0096.MP4': array([], dtype=float64), 'YDXJ0097.MP4': array([], dtype=float64), 'YDXJ0099.MP4': array([], dtype=float64), 'YDXJ0087.MP4': array([], dtype=float64), 'YDXJ0090.MP4': array([], dtype=float64), 'YDXJ0095.MP4': array([], dtype=float64), 'YDXJ0085.MP4': array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
  

In [17]:
for k in neural.keys():
    print(neural[k].shape)

(64, 601, 2)
(64, 601, 2)
(64, 601, 3)
(64, 601, 1)
(64, 601, 3)
(64, 601, 2)
(64, 603, 1)
(64, 600, 2)
(64, 600, 1)
(64, 601, 6)
(64, 601, 1)
(64, 600, 2)
(0,)
(64, 601, 4)
(0,)
(64, 601, 2)
(64, 600, 3)
(64, 601, 2)
(64, 601, 1)
(64, 601, 2)
(64, 600, 7)
(64, 603, 1)
(64, 600, 2)
(64, 600, 2)
